In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.contrib.eager as tfe
# tf.compat.v1.enable_eager_execution()
print(tf.__version__)

import time
import ingestor
import extractor2
import utils
import data
import functools

1.14.0
Tesor Flow Version: 1.14.0 Ingest File
Tesor Flow Version: 1.14.0  Utility File
Tesor Flow Version: 1.14.0 Extactor 2


In [2]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout

Using TensorFlow backend.


In [3]:
params = ingestor.get_params()

In [4]:
train_data = data.get_train_dataset(params)
parser = data.tristan_parser
train_parsed = train_data.map(functools.partial(parser, params=params))

In [7]:
# set features and labels
features = {'goal': [], 'goal_asl': [], 'thms': [], 'thms_hard_negatives': []}
labels = {'tac_id': []}

# iterate over dataset to extract data into arrays. remove 'take' part to iterate over the entire dataset
for raw_record in train_parsed.take(2000):
    fx, lx = raw_record[0], raw_record[1]
    features['goal'].append(fx['goal'])
    features['goal_asl'].append(fx['goal_asl'])
    features['thms'].append(fx['thms'])
    features['thms_hard_negatives'].append(fx['thms_hard_negatives'])
    labels['tac_id'].append(lx['tac_id'])

# instantiate extractor object
ex = extractor2.Extractor(params)

# tokenize goals
temp = ex.tokenize(features['goal'], ex.vocab_table)
#pad all goals to be of length 1000
# goal_list = []
# for j in range(len(temp)):
#         l = len(temp[j])
#         h = tf.pad(temp[j], [[0, 1000-l]], constant_values=0)
#         goal_list.append(h)
# features['goal_ids'] = goal_list
features['goal_ids'] = temp

# tokenize hypotheses. this requires more work since there may be more than one hypothesis
length = len(features['goal'])
features['goal_asl_ids'] = []

for i in range(length):
    temp = ex.tokenize(features['goal_asl'][i], ex.vocab_table)
    #pad all hypotheses to be of length 1000
    hypo_list = []
    for j in range(len(temp)):
        l = len(temp[j])
        h = tf.pad(temp[j], [[0, 1000-l]], constant_values=0)
        hypo_list.append(h)
    features['goal_asl_ids'].append(hypo_list)

del features['goal']
del features['goal_asl']
del features['thms']
del features['thms_hard_negatives']

# # FEATURES
# # 'goal_ids': an array of LENGTH OF DATASET containing arrays which are the tokenized goals of length 1000
# # below is an example of how to access the entire array as numpy arrays
# print(features['goal_ids'][0].numpy())
# print(len(features['goal_ids'][0].numpy()))
# # 'goal_asl_ids': this below is an array of LENGTH OF DATASET containing lists of tokenized hypotheses 
# # where each hypothesis is of length 1000. below is an example of how to access the first hypothesis 
# # from the list of hypotheses corresponding to the first training example
# print(features['goal_asl_ids'][0][0].numpy())
# print(len(features['goal_asl_ids'][0][0].numpy()))

# # LABELS
# # 'tac_id': array containing tactic ids. below is an example of how to access the first tactic
# print(labels['tac_id'][0].numpy())

In [8]:
# features['goal_ids'] is now an array of size 2000 x 1000
features['goal_ids'] = features['goal_ids'].numpy()
print(features['goal_ids'])
print('Number of training examples:', len(features['goal_ids']))
print('Size of training examples:', len(features['goal_ids'][0]))

# features['goal_ids'] is now an array of size 2000 x 1000
labels['tac_id'] = [i.numpy() for i in labels['tac_id']]
print(labels['tac_id'])
print('Number of training examples:', len(labels['tac_id']))

[[2 6 9 ... 0 0 0]
 [2 6 6 ... 0 0 0]
 [2 6 6 ... 0 0 0]
 ...
 [2 6 6 ... 0 0 0]
 [2 6 6 ... 0 0 0]
 [2 6 9 ... 0 0 0]]
Number of training examples: 2000
Size of training examples: 1000
[34, 37, 5, 3, 26, 37, 24, 22, 5, 3, 8, 37, 26, 37, 5, 3, 24, 4, 37, 13, 8, 29, 15, 24, 27, 13, 28, 31, 5, 3, 28, 26, 22, 37, 5, 5, 28, 22, 28, 3, 37, 3, 31, 15, 37, 24, 3, 5, 3, 3, 5, 28, 5, 22, 37, 3, 15, 13, 5, 34, 16, 13, 5, 28, 13, 5, 5, 28, 24, 0, 37, 5, 31, 22, 15, 8, 27, 4, 29, 37, 3, 28, 13, 3, 32, 29, 13, 22, 3, 15, 28, 37, 5, 24, 37, 13, 38, 5, 4, 37, 3, 37, 3, 15, 22, 26, 5, 8, 3, 15, 37, 3, 3, 24, 5, 13, 13, 22, 32, 37, 24, 24, 12, 24, 8, 3, 3, 23, 37, 15, 8, 39, 37, 3, 0, 24, 5, 3, 32, 8, 5, 24, 8, 38, 8, 37, 22, 26, 37, 3, 38, 3, 22, 3, 24, 22, 8, 5, 37, 24, 31, 24, 24, 5, 16, 3, 19, 13, 26, 3, 5, 34, 19, 39, 13, 33, 37, 5, 8, 5, 37, 15, 5, 5, 16, 3, 8, 32, 8, 15, 15, 28, 15, 13, 32, 15, 24, 28, 13, 3, 3, 13, 8, 5, 3, 15, 3, 37, 3, 5, 24, 22, 26, 17, 3, 3, 8, 38, 15, 5, 15, 15, 3, 22, 5, 

In [9]:
# convert to numpy arrays
goals = np.array(features['goal_ids'])
goals

array([[2, 6, 9, ..., 0, 0, 0],
       [2, 6, 6, ..., 0, 0, 0],
       [2, 6, 6, ..., 0, 0, 0],
       ...,
       [2, 6, 6, ..., 0, 0, 0],
       [2, 6, 6, ..., 0, 0, 0],
       [2, 6, 9, ..., 0, 0, 0]], dtype=int32)

In [10]:
# convert to numpy arrays and one-hot encode
a = np.array(labels['tac_id'])
tactics = np.zeros((a.size, a.max()+1))
tactics[np.arange(a.size),a] = 1

In [11]:
tactics

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [31]:
X_train, Y_train = goals, tactics
print(np.shape(X_train))
print(np.shape(Y_train))

(2000, 1000)
(2000, 41)


In [32]:
# save numpy array as csv file
from numpy import asarray
from numpy import savetxt

# save to csv file
savetxt('x_train.csv', X_train, delimiter=',')
savetxt('y_train.csv', Y_train, delimiter=',')